In [5]:
import pandas as pd
import requests
#!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup
import numpy as np
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

**Downloading the data of the New York**

In [6]:
path = 'nyu_2451_34572-geojson.json'

In [7]:
with open(path) as json_data:
    newyork_data = json.load(json_data)

In [8]:
neighborhoods_data = newyork_data['features']

In [9]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [10]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [11]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [12]:
neighborhoods['Borough'].unique()

array(['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island'],
      dtype=object)

**Now we will try to find the best location to open a restaurant in Brooklyn**

In [13]:
brooklyn_data = neighborhoods[neighborhoods['Borough']==('Brooklyn')].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [14]:
print('There are {} Neighbourhoods in Brooklyn' .format(brooklyn_data['Neighborhood'].count()))

There are 70 Neighbourhoods in Brooklyn


**Finding the Coordintaes of Brooklyn and Creating a Map**

In [15]:
address = 'Brooklyn'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


In [16]:
map_newyorkkk = folium.Map(location=[latitude, longitude], zoom_start=11)
map_newyorkkk

In [17]:
for lat, lng, neighborhood in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyorkkk)  
    
map_newyorkkk

**Now using the Fousqaure API to explore the neighbouhoods**

In [18]:
CLIENT_ID = 'GZEGJFOM0NOV1E1ZGDDGI2FJHM15PQBXDMR52C1EGVLDOMZM' # your Foursquare ID
CLIENT_SECRET = 'N24XIQ2GB4FFYCSQXRM4TPCFECY0MMDZFFY1M5JQUD1LOYWS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GZEGJFOM0NOV1E1ZGDDGI2FJHM15PQBXDMR52C1EGVLDOMZM
CLIENT_SECRET:N24XIQ2GB4FFYCSQXRM4TPCFECY0MMDZFFY1M5JQUD1LOYWS


In [19]:
radius = 2000
LIMIT = 100


venues = []

for lat, long, neighborhood in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


**Created a data frame contaning the list of venues in each neighbourhood of Brooklyn**

In [20]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6929, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
1,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
2,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
3,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.622960,-74.031371,Taco Place
4,Bay Ridge,40.625801,-74.030621,Mimi Nails,40.622571,-74.031477,Spa


In [27]:
print('There are {} unique venues in neighbourhoods of Brooklyn'.format(venues_df['VenueCategory'].nunique()))

There are 295 unique venues in neighbourhoods of Brooklyn


**Since we want to open a pizza restaurant, so we will deal with the places of Pizza category only**

In [28]:
pizza_df = venues_df[venues_df['VenueCategory'].str.contains('Pizza')].reset_index(drop=True)
pizza_df.head()


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Bay Ridge,40.625801,-74.030621,Nonno's Pizza,40.621650,-74.031854,Pizza Place
1,Bay Ridge,40.625801,-74.030621,Peppino's,40.629851,-74.028481,Pizza Place
2,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place
3,Bay Ridge,40.625801,-74.030621,Nino's Pizza,40.619763,-74.032630,Pizza Place
4,Bay Ridge,40.625801,-74.030621,Vesuvio Pizzeria & Restaurant,40.632580,-74.027096,Pizza Place


**Let us visulize all the pizza places on the map**

In [29]:
map_pizzaplaces = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, VenueName ,Neighborhood in zip(pizza_df['VenueLatitude'], pizza_df['VenueLongitude'], pizza_df['VenueName'] ,pizza_df['Neighborhood'] ):
    label1 = '{}, {}'.format(VenueName, Neighborhood)
    label = folium.Popup(label1, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_pizzaplaces)

map_pizzaplaces


**Now lets cluster the pizza places, so that we can examine which neighborhoods have less pizza places and which have higher number of pizza places**

In [40]:
temp1 = pizza_df.groupby('Neighborhood').count()
temp1.drop(columns = {'Longitude' ,'VenueName' ,'VenueLatitude', 'VenueLongitude' , 'VenueCategory' } ,inplace=True)
temp1.rename(columns = {'Latitude' : 'Count of Pizza Places'} , inplace=True)
temp1

,Count of Pizza Places
Neighborhood,
Bath Beach,7
Bay Ridge,8
Bedford Stuyvesant,5
Bensonhurst,10
Bergen Beach,3
...,...
Vinegar Hill,3
Weeksville,5
Williamsburg,6


**Now we will use K-means Clustering to clsuter these pizza places**

In [41]:
k = 5

# run k-means clustering
kmeanss = KMeans(n_clusters=k, random_state=0).fit(temp1)

# check cluster labels generated for each row in the dataframe
kmeanss.labels_[0:10] 


array([2, 1, 0, 1, 3, 3, 1, 0, 4, 0], dtype=int32)

**Inserting the column of Cluster labels in the Data**

In [42]:
temp1.insert(0, 'Cluster Labels', kmeanss.labels_)
temp1.head()

,Cluster Labels,Count of Pizza Places
Neighborhood,,
Bath Beach,2,7
Bay Ridge,1,8
Bedford Stuyvesant,0,5
Bensonhurst,1,10
Bergen Beach,3,3


**Now we will merge this Data Frame with a Data Frame , which contains latitue and longitudes of each neighbourhoods**

In [35]:
temp2 = pizza_df[['Neighborhood' , 'Latitude' , 'Longitude']]
temp2.drop_duplicates(subset = 'Neighborhood',inplace=True)
temp2

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Neighborhood,Latitude,Longitude
0,Bay Ridge,40.625801,-74.030621
8,Bensonhurst,40.611009,-73.995180
18,Sunset Park,40.645103,-74.010316
24,Greenpoint,40.730201,-73.954241
27,Gravesend,40.595260,-73.973471
...,...,...,...
401,Dumbo,40.703176,-73.988753
404,Homecrest,40.598525,-73.959185
413,Highland Park,40.681999,-73.890346
424,Madison,40.609378,-73.948415


**Merging the temp and temp1 data frames**

In [43]:
merged = pd.merge(temp1 , temp2 , on='Neighborhood')
merged

,Neighborhood,Cluster Labels,Count of Pizza Places,Latitude,Longitude
0,Bath Beach,2,7,40.599519,-73.998752
1,Bay Ridge,1,8,40.625801,-74.030621
2,Bedford Stuyvesant,0,5,40.687232,-73.941785
3,Bensonhurst,1,10,40.611009,-73.995180
4,Bergen Beach,3,3,40.615150,-73.898556
...,...,...,...,...,...
65,Vinegar Hill,3,3,40.703321,-73.981116
66,Weeksville,0,5,40.675040,-73.930531
67,Williamsburg,2,6,40.707144,-73.958115
68,Windsor Terrace,0,4,40.656946,-73.980073


**Now lets visualize the Clusters on the map**

In [44]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**From above visualization we can say that the neighboorhoods in the cluster label 3 have less number of pizzas**

**Analyzing each cluster cluster**

In [96]:
cluster0 = merged.loc[merged['Cluster Labels']==0]
cluster1 = merged.loc[merged['Cluster Labels']==1]
cluster2 = merged.loc[merged['Cluster Labels']==2]
cluster3 = merged.loc[merged['Cluster Labels']==3]
cluster4 = merged.loc[merged['Cluster Labels']==4]

In [97]:
cluster0.head()

,Neighborhood,Cluster Labels,Count of Pizza Places,Latitude,Longitude
2,Bedford Stuyvesant,0,5,40.687232,-73.941785
7,Brighton Beach,0,4,40.576825,-73.965094
9,Brooklyn Heights,0,5,40.695864,-73.993782
13,Carroll Gardens,0,4,40.680540,-73.994654
18,Crown Heights,0,4,40.670829,-73.943291


In [105]:
cluster1

,Neighborhood,Cluster Labels,Count of Pizza Places,Latitude,Longitude
1,Bay Ridge,1,8,40.625801,-74.030621
3,Bensonhurst,1,10,40.611009,-73.995180
6,Borough Park,1,9,40.633131,-73.990498
10,Brownsville,1,10,40.663950,-73.910235
11,Bushwick,1,8,40.698116,-73.925258
12,Canarsie,1,8,40.635564,-73.902093
23,Dyker Heights,1,9,40.619219,-74.019314
33,Georgetown,1,10,40.623845,-73.916075
39,Homecrest,1,9,40.598525,-73.959185
43,Manhattan Terrace,1,10,40.614433,-73.957438


In [104]:
cluster2

,Neighborhood,Cluster Labels,Count of Pizza Places,Latitude,Longitude
0,Bath Beach,2,7,40.599519,-73.998752
15,Clinton Hill,2,6,40.693229,-73.967843
16,Cobble Hill,2,6,40.687920,-73.998561
20,Ditmas Park,2,6,40.643675,-73.961013
28,Flatbush,2,7,40.636326,-73.958401
29,Flatlands,2,7,40.630446,-73.929113
31,Fort Hamilton,2,7,40.614768,-74.031979
34,Gerritsen Beach,2,7,40.590848,-73.930102
35,Gowanus,2,6,40.673931,-73.994441
41,Madison,2,7,40.609378,-73.948415


In [103]:
cluster3

,Neighborhood,Cluster Labels,Count of Pizza Places,Latitude,Longitude
4,Bergen Beach,3,3,40.615150,-73.898556
5,Boerum Hill,3,1,40.685683,-73.983748
17,Coney Island,3,3,40.574293,-73.988683
21,Downtown,3,3,40.690844,-73.983463
22,Dumbo,3,3,40.703176,-73.988753
24,East Flatbush,3,1,40.641718,-73.936103
26,East Williamsburg,3,3,40.708492,-73.938858
30,Fort Greene,3,3,40.688527,-73.972906
37,Greenpoint,3,3,40.730201,-73.954241
42,Manhattan Beach,3,3,40.577914,-73.943537


In [102]:
cluster4

,Neighborhood,Cluster Labels,Count of Pizza Places,Latitude,Longitude
8,Broadway Junction,4,11,40.677861,-73.903317
14,City Line,4,11,40.678570,-73.867976
19,Cypress Hills,4,14,40.682391,-73.876616
25,East New York,4,12,40.669926,-73.880699
38,Highland Park,4,11,40.681999,-73.890346
51,Ocean Parkway,4,11,40.613060,-73.968367


**From above we can tell that the neighborhoods in the cluster 3 have less number of pizza places**

**Lets visualize Cluster3**

In [106]:
map_pizzaplacesCluster3 = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, VenueName ,Neighborhood in zip(cluster3['Latitude'], cluster3['Longitude'],cluster3['Neighborhood'] ,pizza_df['Neighborhood'] ):
    label1 = '{}'.format(Neighborhood)
    label = folium.Popup(label1, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_pizzaplacesCluster3)

map_pizzaplacesCluster3


**So we have found the neighborhoods in which we can open our new pizza restaurant**


**Further we can find the top 5 neighborhoods with least number of pizza places**

In [116]:
cluster3.sort_values(by ='Count of Pizza Places',inplace=True)


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Neighborhood,Cluster Labels,Count of Pizza Places,Latitude,Longitude
5,Boerum Hill,3,1,40.685683,-73.983748
24,East Flatbush,3,1,40.641718,-73.936103
60,Sea Gate,3,1,40.576375,-74.007873
53,Park Slope,3,2,40.672321,-73.977050
4,Bergen Beach,3,3,40.615150,-73.898556


**Finally , following are the top 5 neighboorhoods having the least number of pizza places in Brooklyn**

In [128]:
cluster3[['Neighborhood']].head(5).reset_index(drop=True)

,Neighborhood
0,Boerum Hill
1,East Flatbush
2,Sea Gate
3,Park Slope
4,Bergen Beach
